In [1]:
import pandas as pd
import numpy as np
from scipy.stats import laplace
import random
import matplotlib.pyplot as plt
from scipy.integrate import quad

df_adult = pd.read_csv("adult.csv")

In [2]:
def plota(scale, a, b):
    dist = laplace(0, scale)
    x = np.linspace(-100, 100,1000)
    plt.plot(x, dist.pdf(x), c='black')
    plt.show()

In [3]:
def consulta1(df,eplison):
    if(eplison == 0): return np.mean(df['age'].values)
    sensibilidade = np.mean(df['age'].values)-(np.sum(df['age'].values)-np.max(df['age'].values))/(df.shape[0]-1)
    x = laplace.rvs(loc = 0, scale = sensibilidade/eplison)
    return [np.mean(df['age'].values) + x,quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]]

In [4]:
def consulta2(df,eplison):
    if(eplison == 0): return np.sum(df['capitalgain'].values)
    sensibilidade = np.max(df['capitalgain'].values)
    x = laplace.rvs(loc = 0, scale = sensibilidade/eplison)
    return [np.sum(df['capitalgain'].values) + x,quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]]

In [5]:
def consulta3(df,eplison):
    if(eplison == 0): return np.mean(df['hoursperweek'].values)
    sensibilidade = np.mean(df['hoursperweek'].values)-(np.sum(df['hoursperweek'].values)-np.max(df['hoursperweek'].values))/(df.shape[0]-1)
    x = laplace.rvs(loc = 0, scale = sensibilidade/eplison)
    return [np.mean(df['hoursperweek'].values) + x,quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]]

In [6]:
def consulta4(df,eplison):
    df_copy = df.copy()
    df_copy = df_copy.replace('>50K', 1)
    df_copy = df_copy.replace('<=50K', 0)
    df_copy['class'] = df_copy['class'].astype(int)
    if(eplison == 0): return np.sum(df_copy['class'].values)
    sensibilidade = np.max(df_copy['class'].values) 
    x = laplace.rvs(loc = 0, scale = sensibilidade/eplison)
    return [np.sum(df_copy['class'].values) + x,quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]]

In [7]:
def consulta(eplison):
    print(str(consulta1(eplison/4)[0]))
    print(str(consulta2(eplison/4)[0]))
    print(str(consulta3(eplison/4)[0]))
    print(str(consulta4(eplison/4)[0]))

In [8]:
def consulta1_teste(df, res,eplison):
    sensibilidade = np.mean(df['age'].values)-(np.sum(df['age'].values)-np.max(df['age'].values))/(df.shape[0]-1)
    x = res-np.mean(df_adult['age'].values)
    return quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]
    

In [9]:
def consulta2_teste(df, res,eplison):
    sensibilidade = np.max(df['capitalgain'].values)
    x = res-np.sum(df_adult['capitalgain'].values)
    return quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]
    

In [10]:
def consulta3_teste(df, res,eplison):
    sensibilidade = np.mean(df['hoursperweek'].values)-(np.sum(df['hoursperweek'].values)-np.max(df['hoursperweek'].values))/(df.shape[0]-1)
    x = res-np.mean(df_adult['hoursperweek'].values)
    return quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]

In [11]:
def consulta4_teste(df, res,eplison):
    df_copy = df_adult.copy()
    df_copy = df_copy.replace('>50K', 1)
    df_copy = df_copy.replace('<=50K', 0)
    df_copy['class'] = df_copy['class'].astype(int)
    df_copy2 = df.copy()
    df_copy2 = df_copy2.replace('>50K', 1)
    df_copy2 = df_copy2.replace('<=50K', 0)
    df_copy2['class'] = df_copy2['class'].astype(int)
    sensibilidade = np.max(df_copy2['class'].values) 
    x = res-np.sum(df_copy['class'].values)
    return quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]

In [12]:
def testa_diffpriv(eplison):
    r = random.randrange(0,df_adult.shape[0]-1)
    df_vizinho = df_adult.copy()
    df_vizinho = df_vizinho.drop(df_vizinho.index[r])
    c1 = consulta1(df_adult,eplison)
    c2 = consulta2(df_adult,eplison)
    c3 = consulta3(df_adult,eplison)
    c4 = consulta4(df_adult,eplison)
    if(
        np.log(c1[1]/consulta1_teste(df_vizinho, c1[0], eplison)) <= eplison and
        np.log(c2[1]/consulta2_teste(df_vizinho, c2[0], eplison)) <= eplison and
        np.log(c3[1]/consulta3_teste(df_vizinho, c3[0], eplison)) <= eplison and
        np.log(c4[1]/consulta4_teste(df_vizinho, c4[0], eplison)) <= eplison
    ):
        return True
    
    print(r)
    print(c1[0])
    print(c1[1])
    print(consulta1_teste(df_vizinho,c1[0],eplison))
    return False
        

In [ ]:
k=1
while(k>0):
    if(not testa_diffpriv(0.1)):break
    print("ok")

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
o

In [ ]:
laplace.pdf(-0.01,0,0.0045)

In [ ]:
np.max(df_copy2['class'].values)